## グルメ用 - 準備

In [15]:
import urllib.request
import urllib.parse
import datetime
import json
import pandas as pd
import googlemaps


# Google Maps Platform Directions API endpoint
endpoint = 'https://maps.googleapis.com/maps/api/directions/json?'
api_key = ''




###インプット欄###
# 出発地、目的地を入力（3つの住所をリストとして受け取る）
addresses = [
    input('候補物件①の住所を入力: '),
    input('候補物件②の住所を入力: '),
    input('候補物件③の住所を入力: ')
]
destinations = input('勤務地の住所を入力: ')  # 共通の目的地


## グルメ用 - 本番

In [ ]:
###"暮らしやすさ - 飲食店"の処理###
# クライアントの生成
gmaps = googlemaps.Client(key=api_key)

# 施設名
place_name = "飲食店"#👈施設名を変えたパターンを用意

# データを格納するためのリスト
data_list = []

# 各住所に対して処理を繰り返す
for address in addresses:
    # 住所から緯度経度を取得
    geocode_result = gmaps.geocode(address)
    location = geocode_result[0]['geometry']['location']

    # 検索範囲の指定 (半径1.2 km)
    radius = 500  # 👈保育園と同じロジック
                  # 👈出力される施設数が500m以内かどうか怪しい

    # 検索結果を取得
    places_result = gmaps.places_nearby(location=location, radius=radius, keyword=place_name, language='ja')

    # 500m半径内の飲食店数を格納する変数
    num_places_within_radius = 0

    # 検索結果をループして500m半径内の施設数をカウント
    for place in places_result['results']:
        # 施設までの所要時間(徒歩)を取得
        distance_matrix_result = gmaps.distance_matrix(location, place['geometry']['location'], mode='walking', language='ja', units='metric')
        distance = distance_matrix_result['rows'][0]['elements'][0]['distance']['value']

        # 半径500m以内の場合にカウント
        if distance <= radius:
            num_places_within_radius += 1
          
    # 施設数をスコアリング👈スコアリングの方法は要相談
    if num_places_within_radius >= 40:
        cnt_score = 100
    elif 30 <= num_places_within_radius < 40:
        cnt_score = 80
    elif 20 <= num_places_within_radius < 30:
        cnt_score = 60
    elif 10 <= num_places_within_radius < 20:
        cnt_score = 40
    else:
        cnt_score = 20
    
    # 最も評価の高い飲食店の評価をスコアリング
    highest_rating = 0
    highest_rating_place = None

    for place in places_result['results']:
        place_id = place['place_id']
        
        # Place Details APIを使用して評価情報を取得
        details_result = gmaps.place(place_id)
        
        if 'result' in details_result:
            rating = details_result['result'].get('rating', 0)
            
            if rating > highest_rating:
                highest_rating = rating
                highest_rating_place= place['name']


    #👈要相談
    # 評価をスコア化（Googleの飲食店スコアはかなりザルなので、5点満点から3を引いた2点満点に換算し50倍することで100点満点にスケーリング）
    if highest_rating_place:
        gourmet_score = (highest_rating - 3) * 50
    else:
        gourmet_score = 0
                
    #カテゴリのスコアリング
    #👈スコアリングの方法は要相談
    category_score = (cnt_score + gourmet_score) / 2
    #部分的に低い点数がある場合に、足し算とするか掛け算とするか
    #足し算⇒加重平均(どちらかに濃淡つけるべきか)
    #調べる段階では施設数、住んでからは距離を重視する⇒どちらかだめならだめ
            
    # 処理結果を辞書に格納
    cat4_data = {'候補物件住所': address,
            'エリア内の飲食店数': num_places_within_radius,
            '最も評価の高い飲食店名': highest_rating_place,
            '最も評価の高い飲食店における評価(星)': highest_rating,
            "飲食店のスコア":category_score,
            "飲食店数スコア":cnt_score,
            "飲食店評価スコア":gourmet_score}

    # データをリストに追加
    data_list.append(cat4_data)

# リストからデータフレームを作成
df4 = pd.DataFrame(data_list)

###"子育てのしやすさ"のアウトプット###
df4
